In [1]:
import cobra
from cobra.io import load_model, read_sbml_model
from cobra.flux_analysis import flux_variability_analysis
import pheflux
import pandas as pd
import scipy
from scipy.stats import pearsonr
import numpy as np
import matplotlib.pyplot as plt
import importlib
import gc
from cobra.sampling import sample
from matplotlib.ticker import MaxNLocator

Welcome to PheFlux ! 



/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [9]:
import sys
teraflux_path = '../..'
if teraflux_path not in sys.path:
    sys.path.append(teraflux_path)
import teraflux

# También puedes usar la función de autoreload para que Jupyter
# recargue tu librería si haces cambios en el archivo .py
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
organism = "Ecoli"
model_dir = "./gems/iJO1366.xml"
model = read_sbml_model(model_dir)
medium_codes = ["ac","fru","gal","glcn","glc","glyc","pyr","succ"]
reaction_codes = {"ac":"EX_ac_e","fru":"EX_fru_e","gal":"EX_gal_e","glcn":"EX_glcn_e","glc":"EX_glc__D_e","glyc":"EX_glyc_e","pyr":"EX_pyr_e","succ":"EX_succ_e","Growth_rate":"BIOMASS_Ec_iJO1366_WT_53p95M"}


In [4]:
# Compute max magnitude for the standard bounds set
max_mag = max(
    max(abs(rxn.lower_bound), abs(rxn.upper_bound))
    for rxn in model.reactions
)

# Define standard bounds
STANDARD_BOUNDS = {0.0, max_mag, -max_mag}

for reaction in model.reactions:
    lb = reaction.lower_bound
    ub = reaction.upper_bound
    if lb==0 and ub==0: print(reaction.id,(lb,ub),reaction.bounds,sep="\t")
    # Only process non-standard bounds
    if lb not in STANDARD_BOUNDS or ub not in STANDARD_BOUNDS:

        # Category 1: always positive
        if lb >= 0 and ub > 0:
            reaction.bounds=(0,max_mag)
        # Category 2: always negative
        elif ub <= 0 and lb < 0:
            reaction.bounds=(-max_mag,0)
        # Category 3: crossing zero
        elif lb < 0 < ub:
            reaction.bounds=(-max_mag,max_mag)
        print(reaction.id,(lb,ub),reaction.bounds,sep="\t")

EX_glc__D_e	(-10.0, 1000.0)	(-1000.0, 1000.0)
EX_cbl1_e	(-0.01, 1000.0)	(-1000.0, 1000.0)
CAT	(0.0, 0.0)	(0.0, 0.0)
ATPM	(3.15, 1000.0)	(0, 1000.0)
DHPTDNR	(0.0, 0.0)	(0.0, 0.0)
DHPTDNRN	(0.0, 0.0)	(0.0, 0.0)
FHL	(0.0, 0.0)	(0.0, 0.0)
SPODM	(0.0, 0.0)	(0.0, 0.0)
SPODMpp	(0.0, 0.0)	(0.0, 0.0)
SUCASPtpp	(0.0, 0.0)	(0.0, 0.0)
SUCFUMtpp	(0.0, 0.0)	(0.0, 0.0)
SUCMALtpp	(0.0, 0.0)	(0.0, 0.0)
SUCTARTtpp	(0.0, 0.0)	(0.0, 0.0)


In [5]:
# Create medium file
for medium_code in medium_codes:
    model = load_model("iJO1366")
    model.reactions.EX_cbl1_e.lower_bound=-1000
    model.reactions.ATPM.lower_bound=0
    model.reactions.EX_ac_e.bounds
    wt_biomass = model.reactions.get_by_id("BIOMASS_Ec_iJO1366_WT_53p95M")
    core_biomass = model.reactions.get_by_id("BIOMASS_Ec_iJO1366_core_53p95M")
    model.objective = wt_biomass
    model.reactions.EX_cbl1_e.lower_bound=-1000
    model.reactions.ATPM.lower_bound=0
    model.reactions.EX_glc__D_e.bounds=(0,1000)
    model.reactions.get_by_id(reaction_codes[medium_code]).bounds=(-10.14,1000)
    model.objective = wt_biomass
    solution=model.optimize()
    print(medium_code)
    print(model.summary())
    with open(f'mediums/Ecoli_Medium_{medium_code}.csv', 'w', newline="\n") as f:
        print("Metabolite_Name", "Reaction_ID", sep="\t", file=f)
        for reaction in model.summary().uptake_flux.index:
            metabolite = reaction.split("_")[1]
            print(metabolite, reaction, sep="\t", file=f)

ac
Objective
1.0 BIOMASS_Ec_iJO1366_WT_53p95M = 0.2644732708848971

Uptake
------
Metabolite     Reaction      Flux  C-Number C-Flux
      ac_e      EX_ac_e     10.14         2 99.98%
     ca2_e     EX_ca2_e   0.00131         0  0.00%
    cbl1_e    EX_cbl1_e 5.898E-05        62  0.02%
      cl_e      EX_cl_e   0.00131         0  0.00%
 cobalt2_e EX_cobalt2_e 6.347E-06         0  0.00%
     cu2_e     EX_cu2_e 0.0001783         0  0.00%
     fe2_e     EX_fe2_e  0.004106         0  0.00%
       h_e       EX_h_e     7.793         0  0.00%
       k_e       EX_k_e   0.04911         0  0.00%
     mg2_e     EX_mg2_e  0.002183         0  0.00%
     mn2_e     EX_mn2_e  0.000174         0  0.00%
    mobd_e    EX_mobd_e 3.623E-05         0  0.00%
     nh4_e     EX_nh4_e     2.779         0  0.00%
     ni2_e     EX_ni2_e 8.119E-05         0  0.00%
      o2_e      EX_o2_e     8.926         0  0.00%
      pi_e      EX_pi_e    0.2452         0  0.00%
     so4_e     EX_so4_e   0.06664         0  0.00%


In [6]:
# Filter the known fluxes to just output the glucose consumption and biomass growth rate
for medium_code in medium_codes:
    
    experimentalFile=f"./experimental/Ecoli/Ecoli_Fluxfile_{medium_code}.csv"
    known_fluxes_filename   =f"./knownFluxes/{organism}/{organism}_knownFluxes_{medium_code}.csv" 
    experimental = pd.read_csv(experimentalFile,index_col=0,sep="\t")
    reactions = [reaction_codes[medium_code],reaction_codes["Growth_rate"] ]

    
    with open(known_fluxes_filename, 'w', newline="\n") as f:
        print("Reaction_ID", "Metabolite_ID", "Reaction_Flux", sep="\t", file=f)
        for Reaction_ID in reactions:
            Metabolite_ID = Reaction_ID
            if Reaction_ID == "BIOMASS_Ec_iJO1366_WT_53p95M":
                Reaction_Flux = experimental.loc["Growth_rate"].Flux 
            else:
                Reaction_Flux = experimental.loc[Reaction_ID].Flux 
            print(Reaction_ID, Metabolite_ID, Reaction_Flux, sep="\t", file=f)

In [ ]:
# Run pheflux

for medium_code in ["ac","gal"]:#medium_codes:
    # Create InputData file
    input_file_name = f"inputData_{organism}_{medium_code}.csv"
    with open(input_file_name, 'w', newline="\n") as f:
        print("Organism\tCondition\tGeneExpFile\tMedium\tNetwork\tKnownFluxes", file=f)    
        Organism = organism
        Condition = medium_code
        GeneExpFile = f"./transcriptomes/{organism}/{organism}_Expfile_{medium_code}.csv"
        Medium = f"./mediums/{organism}_Medium_{medium_code}.csv"
        Network = model_dir 
        KnownFluxes = f"./knownFluxes/{organism}/{organism}_knownFluxes_{medium_code}.csv"
        print(Organism, Condition, GeneExpFile, Medium, Network,KnownFluxes, sep="\t", file=f)

    # Save results
    resultsDir = "./results/pheflux/"
    prefix_log_file = f"{organism}_phe_{medium_code}"
    verbosity = True
    fluxes = pheflux.getFluxes(input_file_name, resultsDir, prefix_log_file, verbosity)

In [10]:
# Run teraflux

for medium_code in medium_codes:
    # Create  InputData file
    input_file_name = f"inputData_{organism}_{medium_code}.csv"
    with open(input_file_name, 'w', newline="\n") as f:
        print("Organism\tCondition\tGeneExpFile\tMedium\tNetwork\tKnownFluxes", file=f)    
        Organism = organism
        Condition = medium_code
        GeneExpFile = f"./transcriptomes/{organism}/{organism}_Expfile_{medium_code}.csv"
        Medium =      f"./mediums/{organism}_Medium_{medium_code}.csv"
        Network = model_dir 
        KnownFluxes = f"./knownFluxes/{organism}/{organism}_knownFluxes_{medium_code}.csv"
        print(Organism, Condition, GeneExpFile, Medium, Network,KnownFluxes, sep="\t", file=f)

    # Save results
    resultsDir = "./results/teraflux/"
    prefix_log_file = f"{organism}_{medium_code}"
    ipoptParams={
            'print_level': 0, # Suppress solver output for cleaner logs
            'sb': 'yes',       # Suppress IPOPT banner
            'tol': 1e-10,       # Overall tolerance (e.g., 1e-8, 1e-10, etc.)
            'acceptable_tol': 1e-10, # Allowable tolerance before giving up
            'max_iter': 6500,   # Increase maximum iterations if needed
            'nlp_scaling_method' : 'none',
            'constr_viol_tol' : 1e-10,
            'nlp_scaling_method': 'gradient-based',
            'hessian_approximation': 'exact',
        }
    fluxes,fr,lagrange = teraflux.getFluxes(input_file_name, resultsDir, prefix_log_file, ipoptParams)

Found 1 conditions to process.

============================= STARTING: Ecoli - ac =============================
[2026/02/07 03:35:39] Loading metabolic model: iJO1366.xml
[2026/02/07 03:35:40] Loading transcriptomic data: Ecoli_Expfile_ac.csv
[2026/02/07 03:35:40] Loading known fluxes: Ecoli_knownFluxes_ac.csv
[2026/02/07 03:35:40] Updating model (bounds, medium)...
Applying medium from: ./mediums/Ecoli_Medium_ac.csv
Starting data pre-processing...
Capping FPKM values at 95th percentile: 23639.3906
Median 'g' value (E_g) for imputation: 3.8060e+03
Pre-processing finished in 0.01 seconds.
[2026/02/07 03:35:41] Running TeraFlux optimization...
Creating stoichiometric constraints...
Adding known flux constraints...
Solving with standard FBA to get an initial guess (x0)...
R_BIOMASS_Ec_iJO1366_core_53p95M
Adding known flux constraints...
R_EX_ac_e -13.584
R_BIOMASS_Ec_iJO1366_WT_53p95M 0.29
{'R_EX_cobalt2_e': inf, 'R_EX_h_e': inf, 'R_EX_k_e': inf, 'R_EX_ac_e': 13.584, 'R_EX_cu2_e': inf, '

In [ ]:
# Run teraflux with all internal fluxes being reversible

for reaction in model.reactions:
    if reaction.id in model.exchanges: continue
    reaction.bounds = (-1000,1000)
cobra.io.write_sbml_model(model,"./gems/iJO1366_allReversible.xml")   

for medium_code in medium_codes:
    # Create  InputData file
    input_file_name = f"inputData_{organism}_{medium_code}.csv"
    with open(input_file_name, 'w', newline="\n") as f:
        print("Organism\tCondition\tGeneExpFile\tMedium\tNetwork\tKnownFluxes", file=f)    
        Organism = organism
        Condition = medium_code
        GeneExpFile = f"./transcriptomes/{organism}/{organism}_Expfile_{medium_code}.csv"
        Medium =      f"./mediums/{organism}_Medium_{medium_code}.csv"
        Network = "./gems/iJO1366_allReversible.xml" 
        KnownFluxes = f"./knownFluxes/{organism}/{organism}_knownFluxes_{medium_code}.csv"
        print(Organism, Condition, GeneExpFile, Medium, Network,KnownFluxes, sep="\t", file=f)

    # Save results
    resultsDir = "./results/teraflux_allReversible/"
    prefix_log_file = f"{organism}_{medium_code}"
    ipoptParams={
            'print_level': 0, # Suppress solver output for cleaner logs
            'sb': 'yes',       # Suppress IPOPT banner
            'tol': 1e-10,       # Overall tolerance (e.g., 1e-8, 1e-10, etc.)
            'acceptable_tol': 1e-10, # Allowable tolerance before giving up
            'max_iter': 6500,   # Increase maximum iterations if needed
            'nlp_scaling_method' : 'none',
            'constr_viol_tol' : 1e-10,
            'nlp_scaling_method': 'gradient-based',
            'hessian_approximation': 'exact',
        }
    fluxes,fr,lagrange = teraflux.getFluxes(input_file_name, resultsDir, prefix_log_file, ipoptParams)

In [ ]:
%pwd